In [1]:
# 필요한 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
# 구글드라이브에 저장해놓은 데이터 불러오기위해 계정연결 ( 구글드라이브 마운트 )
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#필요한 모듈 설치하기
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 44 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595713 sha256=f4ba4df60340a36bf04b10cf85ae4b3eef9291e9d355d0c251482da3e0558571
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 769 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 37.0 MB/s 
     |████████████████████████████████| 3.0 MB 28.8 MB/s 


In [4]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rfu5o6q1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-rfu5o6q1
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=e15ea2083f8d454f0a4903cbfdace0b102ba6f13a684c5b912b26ae36e001a03
  Stored in directory: /tmp/pip-ephem-wheel-cache-7zbvnt0o/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [5]:
#kobert라이브러리에서 많이 쓰이는 함수 불러오기
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers에서 하이퍼파라미터 세팅
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용(권장)
device = torch.device("cuda:0")
#CPU 사용
#device = torch.device("cpu")


#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
#구글 드라이브에 저장되어있던 데이터 불러오기
chatbot_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/감성대화.xlsx')
chatbot_data_new = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/한국어_단발성_대화_데이터셋.xlsx')

In [8]:
#데이터전처리
#당황과 관련된 감정을 필요없으므로 제거
chatbot_data = chatbot_data.loc[(chatbot_data['감정_대분류'] != "당황")]
len(chatbot_data)

8373

In [9]:
#데이터 구조 살펴보기
chatbot_data.sample(n=3)

,연령,성별,상황키워드,감정_대분류,감정_소분류,사람문장1,기계문장1,사람문장2,기계문장2,사람문장3,기계문장3
1282,청소년,남성,학교폭력/따돌림,슬픔,실망한,친하게 지낼 수 있을 거 같던 친구가 오늘 다른 애들이랑 나를 따돌려서 너무 속상해.,친하게 지낼 수 있을 거 같던 친구에게 따돌림을 받아 속상하시군요.,입학 초반에는 인사도 몇 번 나누었는데 언젠가부터 나를 깔보는 것 같더라고.,마음이 아프시겠어요. 지금의 상황을 해결하기 위한 어떤 방법이 있을까요?,강해질 거야. 학교 선생님께 나를 따돌린 애들을 다 말할 거야.,강해지셔서 어떤 아이들도 함부로 대하지 못하게 되셨으면 좋겠어요.
5146,중년,남성,"대인관계(부부, 자녀)",분노,구역질 나는,나는 우리 아이가 문신한 것을 보면 구역질이 나.,문신이 어디에 있나요?,팔뚝에 용 문신을 했더라고.,아드님의 문신이 마음에 안 드시는군요.,NaN,NaN
6031,중년,남성,건강,불안,취약한,난 정신건강에 아주 취약한 것 같아. 그동안 살면서 너무 힘든 상황을 많이 겪어 왔어.,정신건강이 가장 염려되는 부분이시군요. 정신건강을 위해서 노력을 하는 부분은 있나요?,요즘은 최대한 좋은 생각만 하려고 노력을 하고 있어.,긍정적인 생각을 하려고 노력을 하시는군요. 이로 인해 변화는 느끼고 계시나요?,조금 변화가 있는 것 같기도 해. 그래도 걱정은 되긴 하네.,긍정적인 변화를 조금이라도 느끼고 계시지만 여전히 걱정되는 부분이 있군요.


In [10]:
# 감정데이터 정수형(수치)로 바꿔주기 (정수 인코딩)
chatbot_data.loc[(chatbot_data['감정_대분류'] == "불안"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['감정_대분류'] == "상처"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['감정_대분류'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['감정_대분류'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
#chatbot_data.loc[(chatbot_data['감정_대분류'] == "당황"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['감정_대분류'] == "기쁨"), 'Emotion'] = 4  #행복 => 5

In [11]:
# 이중리스트형식으로 data_list에 감정과 텍스트 담기
data_list = []
for q, label in zip(chatbot_data['사람문장1'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)


In [12]:
# data_list에 감정,텍스트 형식 살펴보기
print(data_list[0])
print(data_list[600])
print(data_list[1200])
print(data_list[1800])
print(data_list[2400])
print(data_list[3000])
print(data_list[-1])

['오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.', '4']
['나를 계속 괴롭혔던 애가 너무 미워서 오늘은 그 애 가방 끈을 자르고 도망쳤어.', '2']
['엄마가 나한테 하는 말이 얼마나 큰 상처가 되는지 마음이 너무 아프다.', '3']
['날 따돌리는 친구가 내가 돈이 없어서 수학여행에 가지 못하는 것을 소문 냈어.', '1']
['부모님 덕분에 드디어 신혼 집을 구했어. 그것도 우리 형편에 비해 좋은 집으로.', '4']
['몇 달째 실적이 나지 않아 고민이야.', '0']
['아들이 갑자기 찾아왔는데 손자가 희귀병이래. 너무 두려워.', '0']


In [13]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:
# 훈련,테스트데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

In [16]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
# 파라미터 세팅 ( 문장길이, 모델배치수,에폭수 ...)
max_len = 60
batch_size = 64
warmup_ratio = 0.1
num_epochs = 6
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [26]:
#문장을 토크나이저를 통해서 토큰으로 변환(토큰화)
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [27]:
# 데이터셋 BERT에 맞게 적용
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [28]:
# 텍스트의 토큰화 -> 정수인코딩 결과
data_train[0]

(array([   2, 3501, 3257, 3114, 5760, 1435, 1840, 7101, 6553, 5025, 7799,
        1965, 4213, 6604, 6855,  517,   54, 1185, 3224, 1958, 6371, 4524,
        4634, 7096, 6844,  517,   54,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1], dtype=int32),
 array(28, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32),
 4)

In [29]:
#bert모델에 넣기위해 torch형식으로 변환
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [30]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [31]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [32]:
#optimizer파라미터 세팅
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [33]:
# 옵티마이저설정(AdamW) 추후에 다른 옵티마이저 확인 필요
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [34]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [35]:
#스케줄 설정
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [36]:
# 훈련모델의 정확도 계산함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [37]:
# 훈련데이터 형식 확인 
train_dataloader

In [38]:
#데이터 학습모델을 통해 학습 and 테스트
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.6063991785049438 train acc 0.28125
epoch 1 train acc 0.39008387445887444


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 5 test acc 0.6899497694334651


  0%|          | 0/99 [00:00<?, ?it/s]

epoch 6 batch id 1 loss 0.236735999584198 train acc 0.921875
epoch 6 train acc 0.8809974747474747


  0%|          | 0/33 [00:00<?, ?it/s]

epoch 6 test acc 0.6830327733860342


In [44]:
#classification_report 만들기위해 데이터 전처리
x_pred = []
x_label = []
y_test = []
for i in dataset_test:
  x_pred.append(int(i[1]))
  x_label.append(str(i[0]))


test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)
model.eval()

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]
    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()
        
        y_test.append(np.argmax(logits))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [48]:
# 데이터셋 각 라벨에 대한 정확도 측정
#y_predicted = bilstm_model.predict([X_test])
#pred_tags = f1score.sequences_to_tags(y_predicted)
#test_tags = f1score.sequences_to_tags(y_test)

#print(classification_report(test_tags, pred_tags))

#confusion_matrix 와 classification_report를 통해 결과값 정리
from sklearn.metrics import confusion_matrix, classification_report

df = pd.DataFrame(columns=['Labels Data','Predicted Labels', 'Actual Labels'])
df['Labels Data'] = x_label
df['Predicted Labels'] = x_pred
df['Actual Labels'] = y_test

df.head(10)



In [50]:
print(classification_report(x_pred, y_test,target_names=['불안','상처','분노','슬픔','기쁨']))

              precision    recall  f1-score   support

          불안       0.62      0.74      0.67       397
          상처       0.66      0.58      0.62       449
          분노       0.66      0.60      0.63       393
          슬픔       0.60      0.61      0.60       433
          기쁨       0.89      0.90      0.89       422

    accuracy                           0.68      2094
   macro avg       0.68      0.69      0.68      2094
weighted avg       0.69      0.68      0.68      2094



In [53]:
# 개별 문장을 넣어볼수 있는 (예측하는) 함수
y_test = []
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            #print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("불안이")
            elif np.argmax(logits) == 1:
                test_eval.append("상처가")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("행복이")
            elif np.argmax(logits) == 5:
                test_eval.append("당황이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
            
        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")


In [54]:
# 문장입력기
sentence = input("하고싶은 말을 입력해주세요 : ")
predict(sentence)
print("\n")

하고싶은 말을 입력해주세요 : 상사한테 혼나서 너무 슬퍼


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 슬픔이 느껴집니다.


